In [55]:
import numpy as np
import pandas as pd
import sympy
from ast import literal_eval
import os, time

# Random Functions

In [56]:
RESULT_DIR = "final_results"
benchmark = "manual"

In [57]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = None
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
            
            if tool_name == 'volesti':
                try:
                    df['volesti']['details'] = df['volesti']['details'].apply(literal_eval)
                                        
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['output'] for i in x]))
                    df['volesti']['output'] = list(zip(df['volesti']['np'].apply(np.mean), df['volesti']['np'].apply(np.std)))            
                
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['time'] for i in x]))
                    df['volesti']['time'] = df['volesti']['np'].apply(np.mean)            
                
                except:
                    pass
            print(tool_name, df[tool_name].columns)
            
    except Exception as e:
        pass

volesti Index(['bechmark', 'formula', 'bounds', 'index', 'output', 'error', 'time',
       'details', 'np'],
      dtype='object')
faza Index(['bechmark', 'formula', 'index', 'output', 'error', 'time', 'details'], dtype='object')
latte Index(['bechmark', 'formula', 'bounds', 'index', 'output', 'error', 'time',
       'details'],
      dtype='object')


/home/des/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/des/venv/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/des/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [58]:
tools = sorted(list(df.keys()))
tool0 = tools[0]
tool1 = tools[1]

df_res = df[tool0].join(df[tool1], on='index', lsuffix=f'_{tool0}', rsuffix=f'_{tool1}', how='outer')

for t in tools[2:]:
    df_res = df_res.join(df[t].add_suffix(f'_{t}'), on='index', how='outer')


In [59]:
df_res = df_res.sort_values('index_volesti')
selected_cols = [f'formula_volesti', f'bounds_volesti']


for t in tools:
    selected_cols.append(f"output_{t}")
    df_res[f"output_{t}"] = df_res[f"output_{t}"].fillna(df_res[f"error_{t}"]).fillna('TBD').astype(str).str[:1000]
    selected_cols.append(f"time_{t}")
    df_res[f"time_{t}"] = df_res[f"time_{t}"].fillna(0).apply(lambda x:round(x, 3))
    
df_res = df_res[selected_cols]
df_res['formula_volesti'] = df_res['formula_volesti'].apply(sympy.parse_expr)
df_res

,formula_volesti,bounds_volesti,output_faza,time_faza,output_latte,time_latte,output_volesti,time_volesti
0,x,"[[0, 1]]","(0.4429687500000001, 0.5414062500000002)",1.801,0.495,0.046,"(0.5151958000000001, 0.018589615982047614)",0.062
1,x**2,"[[0, 1]]","(0.2919921875, 0.3916015625)",2.998,0.3333333333333333,0.056,"(0.32834620000000003, 0.02313598325033971)",0.056
2,x**3,"[[0, 1]]","(0.203125, 0.302734375)",18.310,0.25,0.056,"(0.2444998, 0.012170452841205211)",0.063
3,x**3 + x**2 + x + 1,"[[0, 1]]","(2.033935546875, 2.1337890625)",30.536,2.083333333333333,0.077,"(2.103787, 0.12710389868528818)",0.077
4,x**4 + x**3 + x**2 + x + 1,"[[0, 1]]","(2.2339630126953125, 2.33367919921875)",185.478,2.283333333333333,0.065,"(2.3580829999999997, 0.11347025919156088)",0.061
5,1/x,"[[0.1, 1]]","(2.2532958984375, 2.3532028198242188)",13.755,Memory limit exceeded (maybe),0.109,"(2.2516629999999997, 0.16795978108166246)",0.060
6,1/(x + y),"[[0.1, 1], [0.1, 1]]","(0.8045571327208667, 0.9045425891875298)",306.383,Not a monomial: (x + y),0.019,"(0.8608950999999999, 0.04969340924800792)",0.063
7,1/(x*y + x + y),"[[0.1, 1], [0.1, 1]]","(0.6644693102154389, 0.7644645145960148)",2024.907,Not a monomial: (x + y + (x * y)),0.023,"(0.7262099, 0.0389765557212281)",0.064
8,1/(x + y + z),"[[0.1, 1], [0.1, 1], [0.1, 1]]","(0.43449236869801694, 0.5344923090936162)",6560.093,Not a monomial: (x + y + z),0.022,"(0.49478150000000004, 0.024827443345016412)",0.066
9,1/(x**2 - 1),"[[1.01, 2]]","(2.053851511941105, 2.1538314534657927)",162.084,Not a monomial: (-1.0 + (x ^ 2.0)),0.030,"(2.13001, 0.15017606187405494)",0.060


In [60]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)